In [3]:
import netCDF4 as nc
import os
import matplotlib.pyplot as plt
from netCDF4 import Dataset as netcdf_dataset
import numpy as np
import pandas as pd
import xarray as xr
import time
import geopandas 
import dask.dataframe as dd
from math import radians, sin, cos, asin, sqrt
from shapely.geometry import Polygon
from shapely.geometry import Point
import scipy.spatial as spatial
from shapely import wkt
import shapely
import pyproj
pyproj.datadir.set_data_dir("/opt/anaconda3/envs/geo_env/share/proj")
import numpy_financial as npf
from pandas.api.types import CategoricalDtype
from numpy import savetxt
from numpy import loadtxt

## Read and prepare wind speed data

In [10]:
#read NASA datafile
ds=xr.open_mfdataset('/Users/alexandrapopova/MERRA-2-2019/*.nc', parallel=True)

In [12]:
#convert to dataframe
source_df= ds.to_dataframe()

In [14]:
#reset names and indices 
source_df=source_df.rename(columns={"SPEED": "Wind_Speed"})
source_df=source_df.reset_index()

In [16]:
#split datetime into date and time to aggregate next
source_df['newdate']=source_df['time'].dt.date
source_df['newtime']=source_df['time'].dt.time

In [24]:
#mark seasons
def get_season (date):
    month=date.month
    if month<3 or month>8:
        season="aw"
    else:
        season="ss"
    return season
source_df['season']=source_df['newdate'].apply(lambda x: get_season(x))

In [26]:
#aggregate
aggr_df=source_df.groupby(['lat','lon','newtime','season'] ).agg({'Wind_Speed': ['mean', 'max', np.std, 'count']})

In [31]:
aggr_df=aggr_df.reset_index()

In [32]:
z=1.645 # 90% confidence interval

In [33]:
#calculate intervals
aggr_df['Wind_Speed_Min']=aggr_df['Wind_Speed']['mean']-z*aggr_df['Wind_Speed']['std']/np.sqrt(aggr_df['Wind_Speed']['count'])

aggr_df['Wind_Speed_Max']=aggr_df['Wind_Speed']['mean']+z*aggr_df['Wind_Speed']['std']/np.sqrt(aggr_df['Wind_Speed']['count'])

In [36]:
aggr_df=aggr_df.drop(columns='Wind_Speed')

/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [35]:
aggr_df=aggr_df.rename(columns={"newtime": "time"})

In [38]:
aggr_df.columns = aggr_df.columns.droplevel(-1)

In [39]:
aggr_df.to_csv('Wind_Speed_Dataframe.csv')

In [6]:
#aggr_df=pd.read_csv('Wind_Speed_Dataframe.csv')

## Create polygons

In [40]:
#increase the resolution x4
half_lat=0.5*0.5
half_lon=0.625*0.5

aggr_df['coords']=list(zip(aggr_df['lon'],aggr_df['lat']))
aggr_df_new_rows_1=aggr_df.copy()
aggr_df_new_rows_2=aggr_df.copy()
aggr_df_new_rows_3=aggr_df.copy()

aggr_df['geometry']=aggr_df['coords'].apply(lambda x: Polygon([(x[0], x[1]), (x[0]-half_lon, x[1]), (x[0]-half_lon, x[1]+half_lat), (x[0], x[1]+half_lat)]))
aggr_df_new_rows_1['geometry']=aggr_df_new_rows_1['coords'].apply(lambda x: Polygon([(x[0], x[1]), (x[0], x[1]+half_lat), (x[0]+half_lon, x[1]+half_lat), (x[0]+half_lon, x[1])]))
aggr_df_new_rows_2['geometry']=aggr_df_new_rows_2['coords'].apply(lambda x: Polygon([(x[0], x[1]), (x[0]+half_lon, x[1]), (x[0]+half_lon, x[1]-half_lat), (x[0], x[1]-half_lat)]))
aggr_df_new_rows_3['geometry']=aggr_df_new_rows_3['coords'].apply(lambda x: Polygon([(x[0], x[1]), (x[0], x[1]-half_lat), (x[0]-half_lon, x[1]-half_lat), (x[0]-half_lon, x[1])]))

aggr_df=aggr_df.append(aggr_df_new_rows_1).append(aggr_df_new_rows_2).append(aggr_df_new_rows_3)

## Calculate turbine energy production

In [41]:
def Vestas_Power(x):
    if x < 3:
        return 0
    else:
        if (x>=3) and (x<4):
            return (149*x-412) #source - power curves
        else:
            if (x>=4) and (x<12):
                return (463.2*x-1755.5) #source - power curves
            else: 
                if (x>=12) and (x<=22):
                    return 3450 #source - power curves
                else:
                    return 0

In [42]:
def Nordex_Power(x):
    if x < 3:
        return 0
    else:
        if (x>=3) and (x<5):
            return (83*x-248) #source - power curves
        else:
            if (x>=5) and (x<14):
                return (281.3*x-1126.9) #source - power curves
            else: 
                if (x>=14) and (x<=25):
                    return 2500 #source - power curves
                else:
                    return 0

In [43]:
def Enercon_Power(x):
    if x < 3:
        return 0
    else:
        if (x>=3) and (x<5):
            return (27.5*x-80) #source - power curves
        else:
            if (x>=5) and (x<14):
                return (88.8*x-363.04) #source - power curves
            else: 
                if (x>=14) and (x<=34):
                    return 810 #source - power curves
                else:
                    return 0

In [44]:
aggr_df['Vestas_Power_Min']=aggr_df['Wind_Speed_Min'].apply(Vestas_Power)
aggr_df['Vestas_Power_Max']=aggr_df['Wind_Speed_Max'].apply(Vestas_Power)

In [45]:
aggr_df['Nordex_Power_Min']=aggr_df['Wind_Speed_Min'].apply(Nordex_Power)
aggr_df['Nordex_Power_Max']=aggr_df['Wind_Speed_Max'].apply(Nordex_Power)

In [46]:
aggr_df['Enercon_Power_Min']=aggr_df['Wind_Speed_Min'].apply(Enercon_Power)
aggr_df['Enercon_Power_Max']=aggr_df['Wind_Speed_Max'].apply(Enercon_Power)

In [47]:
aggr_df['Vestas_Energy_Min_MWh']=aggr_df['Vestas_Power_Min']/1000
aggr_df['Vestas_Energy_Max_MWh']=aggr_df['Vestas_Power_Max']/1000
aggr_df['Nordex_Energy_Min_MWh']=aggr_df['Nordex_Power_Min']/1000
aggr_df['Nordex_Energy_Max_MWh']=aggr_df['Nordex_Power_Max']/1000
aggr_df['Enercon_Energy_Min_MWh']=aggr_df['Enercon_Power_Min']/1000
aggr_df['Enercon_Energy_Max_MWh']=aggr_df['Enercon_Power_Max']/1000

In [54]:
#add hours
aggr_df['hour']=pd.Series(np.tile([1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11,12,12,13,13,14,14,15,15,16,16,17,17,18,18,19,19,20,20,21,21,22,22,23,23,24,24],int(len(aggr_df)/48)))

In [49]:
#takes out matching errors
aggr_df['lon']=aggr_df['lon'].round(3)

In [51]:
#aggregate
aggr_df=aggr_df.drop(columns=['time','Wind_Speed_Min','Wind_Speed_Max', 'Vestas_Power_Min', 'Vestas_Power_Max','Nordex_Power_Min','Nordex_Power_Max','Enercon_Power_Min','Enercon_Power_Max'])
aggr_df=aggr_df.sort_values(by=['lat', 'lon'])

In [56]:
aggr_df.to_csv('Wind_Energy_Polygons_Dataframe.csv')

In [105]:
#aggr_df = pd.read_csv('Wind_Energy_Polygons_Dataframe.csv')

## Append solar data

In [106]:
solar_df=pd.read_csv('Solar_Power.csv')

In [107]:
#create identifier for lookup
solar_df['coords-season-hour']=solar_df['coords']+"-"+solar_df['season']+"-"+solar_df['hour'].astype(str)

In [126]:
#create identifier for lookup
aggr_df['coords-season-hour']=aggr_df['coords'].astype(str)+"-"+aggr_df['season']+"-"+aggr_df['hour'].astype(str)

In [109]:
#join minimum solar energy value to the wind energy dataset
aggr_df['SolarPV_Energy_Min_W']= aggr_df['coords-season-hour'].map(solar_df.set_index('coords-season-hour')['P_Min'].to_dict())

In [110]:
#join maximum solar energy value to the wind energy dataset
aggr_df['SolarPV_Energy_Max_W']= aggr_df['coords-season-hour'].map(solar_df.set_index('coords-season-hour')['P_Max'].to_dict())

In [111]:
#fill the areas where no solar energy data was found (e.g. next to water)
aggr_df['SolarPV_Energy_Min_W'] = aggr_df['SolarPV_Energy_Min_W'].fillna(0)

In [112]:
#fill the areas where no solar energy data was found (e.g. next to water)
aggr_df['SolarPV_Energy_Max_W'] = aggr_df['SolarPV_Energy_Max_W'].fillna(0)

In [113]:
aggr_df['SolarPV_Energy_Min_MWh']=aggr_df['SolarPV_Energy_Min_W']/1000000
aggr_df['SolarPV_Energy_Max_MWh']=aggr_df['SolarPV_Energy_Max_W']/1000000

In [114]:
aggr_df=aggr_df.drop(columns=['SolarPV_Energy_Min_W','SolarPV_Energy_Max_W'])

## Create geodataframe for filtering

In [79]:
filter_df=aggr_df.loc[(aggr_df['hour'] == 1)&(aggr_df['season'] == 'ss')]
filter_df=filter_df.drop(columns=['lat','lon','season','Vestas_Energy_Min_MWh','Vestas_Energy_Max_MWh','Nordex_Energy_Min_MWh','Nordex_Energy_Max_MWh','Enercon_Energy_Min_MWh','Enercon_Energy_Max_MWh','SolarPV_Energy_Min_MWh','SolarPV_Energy_Max_MWh', 'hour'])

In [82]:
filter_df['centroid']=filter_df['geometry'].apply(lambda x: x.centroid)

In [83]:
filter_gdf = geopandas.GeoDataFrame(filter_df, geometry='geometry')
filter_gdf=filter_gdf.set_crs("EPSG:4326")

## Intersect with countries

In [84]:
countries_df = geopandas.read_file('/Users/alexandrapopova/Downloads/ref-countries-2020-60m.geojson/CNTR_RG_60M_2020_4326.geojson')
CountryList = ['GBR', 'AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE']
countries_df = countries_df[countries_df['ISO3_CODE'].isin(CountryList)]

In [85]:
countries_df=countries_df.drop(columns=['id','CNTR_ID','CNTR_NAME', 'NAME_ENGL','FID'])

In [87]:
results=geopandas.overlay(filter_gdf, countries_df, how='intersection')

In [88]:
results.to_csv('Polygons+Countries_Dataframe.csv')

In [31]:
#results_df = pd.read_csv('Polygons+Countries_Dataframe.csv')

In [32]:
#results_df['geometry']=results_df['geometry'].apply(wkt.loads)

In [33]:
#results= geopandas.GeoDataFrame(results_df, geometry='geometry')

In [34]:
#results=results.drop(columns=['Unnamed: 0'])

In [211]:
results.drop(columns=['coords']).to_file("polygons+countries.geojson", driver="GeoJSON")  

## Intersect with Land use

In [3]:
#land use files are split into multiple due to large datasize - loaded here separately
land_use_1 = geopandas.read_file('land_use_1_CDDA.shp')

In [14]:
land_use_2 = geopandas.read_file('land_use_2_CDDA.shp')

In [8]:
land_use_3 = geopandas.read_file('land_use_3_CDDA.shp')

In [9]:
land_use_4 = geopandas.read_file('land_use_4_CDDA.shp')

In [10]:
land_use_5 = geopandas.read_file('land_use_5_CDDA.shp')

In [18]:
land_use_6 = geopandas.read_file('land_use_6_CDDA.shp')

In [19]:
land_use_7 = geopandas.read_file('land_use_7_CDDA.shp')

In [20]:
land_use_8 = geopandas.read_file('land_use_8_CDDA.shp')

In [21]:
land_use_9 = geopandas.read_file('land_use_9_CDDA.shp')

In [22]:
land_use_10 = geopandas.read_file('land_use_10_CDDA.shp')

In [23]:
land_use_11 = geopandas.read_file('land_use_11_CDDA.shp')

In [24]:
#intersects energy data cells with land use polygons
results_landuse_1=geopandas.overlay(results, land_use_1, how='intersection')
results_landuse_2=geopandas.overlay(results, land_use_2, how='intersection')
results_landuse_3=geopandas.overlay(results, land_use_3, how='intersection')
results_landuse_4=geopandas.overlay(results, land_use_4, how='intersection')
results_landuse_5=geopandas.overlay(results, land_use_5, how='intersection')
results_landuse_6 =geopandas.overlay(results, land_use_6, how='intersection')
results_landuse_7=geopandas.overlay(results, land_use_7, how='intersection')
results_landuse_8=geopandas.overlay(results, land_use_8, how='intersection')
results_landuse_9=geopandas.overlay(results, land_use_9, how='intersection')
results_landuse_10=geopandas.overlay(results, land_use_10, how='intersection')
results_landuse_11=geopandas.overlay(results, land_use_11, how='intersection')

In [27]:
#save results
results_landuse_1.to_file("land_use_polygons_simplify.shp")

In [3]:
#results_landuse_1=geopandas.read_file('land_use_polygons_simplify.shp')

In [28]:
results_landuse_2.to_file("land_use_polygons_2_simplify.shp")

In [4]:
#results_landuse_2=geopandas.read_file('land_use_polygons_2_simplify.shp')

In [ ]:
results_landuse_3.to_file("land_use_polygons_3_simplify.shp")

In [5]:
#results_landuse_3=geopandas.read_file('land_use_polygons_3_simplify.shp')

In [ ]:
results_landuse_4.to_file("land_use_polygons_4_simplify.shp")

In [6]:
#results_landuse_4=geopandas.read_file('land_use_polygons_4_simplify.shp')

In [ ]:
results_landuse_5.to_file("land_use_polygons_5_simplify.shp")

In [7]:
#results_landuse_5=geopandas.read_file('land_use_polygons_5_simplify.shp')

In [ ]:
results_landuse_6.to_file("land_use_polygons_6_simplify.shp")

In [8]:
#results_landuse_6=geopandas.read_file('land_use_polygons_6_simplify.shp')

In [ ]:
results_landuse_7.to_file("land_use_polygons_7_simplify.shp")

In [10]:
#results_landuse_7=geopandas.read_file('land_use_polygons_7_simplify.shp')

In [ ]:
results_landuse_8.to_file("land_use_polygons_8_simplify.shp")

In [9]:
#results_landuse_8=geopandas.read_file('land_use_polygons_8_simplify.shp')

In [ ]:
results_landuse_9.to_file("land_use_polygons_9_simplify.shp")

In [11]:
#results_landuse_9=geopandas.read_file('land_use_polygons_9_simplify.shp')

In [ ]:
results_landuse_10.to_file("land_use_polygons_10_simplify.shp")

In [12]:
#results_landuse_10=geopandas.read_file('land_use_polygons_10_simplify.shp')

In [ ]:
results_landuse_11.to_file("land_use_polygons_11_simplify.shp")

In [13]:
#results_landuse_11=geopandas.read_file('land_use_polygons_11_simplify.shp')

In [14]:
#result of the overlay is multipolygons - break into smaller polygons

def breakintopolygons(results_landuse, results_landuse_polygons):
    for i in range(len(results_landuse)):
        if (type(results_landuse.geometry.iloc[i])==Polygon):
            results_landuse_polygons=results_landuse_polygons.append(results_landuse.iloc[i])
        else:
            for j in range (len(results_landuse.geometry.iloc[i])):
                if (results_landuse.geometry.iloc[i][j].area<0.0001):
                    continue
                else:
                    row=results_landuse.iloc[i]
                    row['geometry']=results_landuse.geometry.iloc[i][j]
                    results_landuse_polygons=results_landuse_polygons.append(row)
    return results_landuse_polygons

In [16]:
results_landuse_polygons_1 = geopandas.GeoDataFrame(columns=results_landuse_1.columns)
results_landuse_polygons_1=breakintopolygons(results_landuse_1, results_landuse_polygons_1)

In [17]:
results_landuse_polygons_2 = geopandas.GeoDataFrame(columns=results_landuse_2.columns)
results_landuse_polygons_2=breakintopolygons(results_landuse_2, results_landuse_polygons_2)

In [18]:
results_landuse_polygons_3 = geopandas.GeoDataFrame(columns=results_landuse_3.columns)
results_landuse_polygons_3=breakintopolygons(results_landuse_3, results_landuse_polygons_3)

In [19]:
results_landuse_polygons_4 = geopandas.GeoDataFrame(columns=results_landuse_4.columns)
results_landuse_polygons_4=breakintopolygons(results_landuse_4, results_landuse_polygons_4)

In [20]:
results_landuse_polygons_5 = geopandas.GeoDataFrame(columns=results_landuse_5.columns)
results_landuse_polygons_5=breakintopolygons(results_landuse_5, results_landuse_polygons_5)

In [21]:
results_landuse_polygons_6 = geopandas.GeoDataFrame(columns=results_landuse_6.columns)
results_landuse_polygons_6=breakintopolygons(results_landuse_6, results_landuse_polygons_6)

In [22]:
results_landuse_polygons_7 = geopandas.GeoDataFrame(columns=results_landuse_7.columns)
results_landuse_polygons_7=breakintopolygons(results_landuse_7, results_landuse_polygons_7)

In [23]:
results_landuse_polygons_8 = geopandas.GeoDataFrame(columns=results_landuse_8.columns)
results_landuse_polygons_8=breakintopolygons(results_landuse_8, results_landuse_polygons_8)

In [24]:
results_landuse_polygons_9 = geopandas.GeoDataFrame(columns=results_landuse_9.columns)
results_landuse_polygons_9=breakintopolygons(results_landuse_9, results_landuse_polygons_9)

In [37]:
results_landuse_polygons_10 = geopandas.GeoDataFrame(columns=results_landuse_10.columns)
results_landuse_polygons_10=breakintopolygons(results_landuse_10, results_landuse_polygons_10)

KeyboardInterrupt: 

In [ ]:
results_landuse_polygons_11 = geopandas.GeoDataFrame(columns=results_landuse_10.columns)
results_landuse_polygons_11=breakintopolygons(results_landuse_10, results_landuse_polygons_10)

In [40]:
results_landuse_1=results_landuse_polygons_1
results_landuse_2=results_landuse_polygons_2
results_landuse_3=results_landuse_polygons_3
results_landuse_4=results_landuse_polygons_4
results_landuse_5=results_landuse_polygons_5
results_landuse_6=results_landuse_polygons_6
results_landuse_7=results_landuse_polygons_7
results_landuse_8=results_landuse_polygons_8
results_landuse_9=results_landuse_polygons_9

## Add lines cost

In [210]:
df = pd.read_csv('substations.csv')
substations_gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.lon, df.lat))

In [211]:
#substations_gdf.to_file('substations.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/448186792.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  substations_gdf.to_file('substations.shp')


In [39]:
#create a KDTree to find the nearest neighbours
point_array=np.empty((len(substations_gdf),2))
for i in range(len(substations_gdf)):
    point_array[i,0]=substations_gdf['geometry'][i].x
    point_array[i,1]=substations_gdf['geometry'][i].y
point_tree = spatial.cKDTree(point_array)

In [41]:
#distance is measured to centroids - find centroids
results_landuse_1['centroid']=results_landuse_1['geometry'].apply(lambda x: x.centroid)
results_landuse_2['centroid']=results_landuse_2['geometry'].apply(lambda x: x.centroid)
results_landuse_3['centroid']=results_landuse_3['geometry'].apply(lambda x: x.centroid)
results_landuse_4['centroid']=results_landuse_4['geometry'].apply(lambda x: x.centroid)
results_landuse_5['centroid']=results_landuse_5['geometry'].apply(lambda x: x.centroid)
results_landuse_6['centroid']=results_landuse_6['geometry'].apply(lambda x: x.centroid)
results_landuse_7['centroid']=results_landuse_7['geometry'].apply(lambda x: x.centroid)
results_landuse_8['centroid']=results_landuse_8['geometry'].apply(lambda x: x.centroid)
results_landuse_9['centroid']=results_landuse_9['geometry'].apply(lambda x: x.centroid)
results_landuse_10['centroid']=results_landuse_10['geometry'].apply(lambda x: x.centroid)
results_landuse_11['centroid']=results_landuse_11['geometry'].apply(lambda x: x.centroid)

In [42]:
# Haversine formula for kilometer distance between two lat/long points
def haversine_dist_from_coords(lat1, lon1, lat2, lon2):
    # The math module contains a function named radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
    # calculate and return the result
    return c * r

In [43]:
def lines_length (point, lines_df):
    lines_df['distance']=lines_df['geometry'].apply(lambda z: haversine_dist_from_coords(z.x, z.y, point.x, point.y))
    distance=(lines_df[['distance']].min())
    return distance

In [15]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

In [44]:
def write_lines_cost (results_landuse_polygons):
    results_landuse_polygons['lines_length']=np.nan
    results_landuse_polygons['lines_cost']=np.nan
    for p in range (len(results_landuse_polygons)):
        nearest=(point_tree.query_ball_point([results_landuse_polygons['centroid'].iloc[p].x, results_landuse_polygons['centroid'].iloc[p].y], 1))
        if (len(nearest)==0):
            nearest=(point_tree.query_ball_point([results_landuse_polygons['centroid'].iloc[p].x, results_landuse_polygons['centroid'].iloc[p].y], 2))
        if (len(nearest)==0):
            nearest=(point_tree.query_ball_point([results_landuse_polygons['centroid'].iloc[p].x, results_landuse_polygons['centroid'].iloc[p].y], 20))
        gdf_nearest_subs = geopandas.GeoDataFrame(columns=substations_gdf.columns)
        gdf_nearest_subs=substations_gdf.iloc[(nearest)]
        results_landuse_polygons['lines_length'].iloc[p]=lines_length(results_landuse_polygons['centroid'].iloc[p], gdf_nearest_subs)
        price=95 #thousands euros per km
        results_landuse_polygons['lines_cost']=results_landuse_polygons['lines_length']*price
    return results_landuse_polygons

In [47]:
results_landuse_1=write_lines_cost(results_landuse_1)

In [48]:
results_landuse_2=write_lines_cost(results_landuse_2)

In [49]:
results_landuse_3=write_lines_cost(results_landuse_3)

In [50]:
results_landuse_4=write_lines_cost(results_landuse_4)

In [51]:
results_landuse_5=write_lines_cost(results_landuse_5)

In [52]:
results_landuse_6=write_lines_cost(results_landuse_6)

In [53]:
results_landuse_7=write_lines_cost(results_landuse_7)

In [54]:
results_landuse_8=write_lines_cost(results_landuse_8)

In [55]:
results_landuse_9=write_lines_cost(results_landuse_9)

In [56]:
results_landuse_10=write_lines_cost(results_landuse_10)

In [57]:
results_landuse_11=write_lines_cost(results_landuse_11)

In [58]:
"""
results_landuse_1.to_csv('results_landuse+costs_1.csv')
results_landuse_2.to_csv('results_landuse+costs_2.csv')
results_landuse_3.to_csv('results_landuse+costs_3.csv')
results_landuse_4.to_csv('results_landuse+costs_4.csv')
results_landuse_5.to_csv('results_landuse+costs_5.csv')
results_landuse_6.to_csv('results_landuse+costs_6.csv')
results_landuse_7.to_csv('results_landuse+costs_7.csv')
results_landuse_8.to_csv('results_landuse+costs_8.csv')
results_landuse_9.to_csv('results_landuse+costs_9.csv')
results_landuse_10.to_csv('results_landuse+costs_10.csv')
results_landuse_11.to_csv('results_landuse+costs_11.csv')
"""

## Add distance to roads

In [59]:
roads_gdf = geopandas.read_file('/Users/alexandrapopova/Downloads/europe-road.geojson')
roads_gdf=roads_gdf[['rtn','geometry']]

In [60]:
#same methodology as for the lines distance
def road_distance (point, road_df):
    road_df['closest_point']=road_df['geometry'].apply(lambda x: x.interpolate(x.project(point)))
    road_df['distance']=road_df['closest_point'].apply(lambda z: haversine_dist_from_coords(z.x, z.y, point.x, point.y))
    distance=(road_df[['distance']].min())
    return distance

In [61]:
roads_gdf['centroid']=roads_gdf['geometry'].apply(lambda x: x.centroid)

In [62]:
centroid_array=np.empty((len(roads_gdf),2))
for i in range(len(roads_gdf)):
    centroid_array[i,0]=roads_gdf['centroid'][i].x
    centroid_array[i,1]=roads_gdf['centroid'][i].y
point_tree = spatial.cKDTree(centroid_array)

In [63]:
def distance_to_roads(results_landuse_polygons):
    results_landuse_polygons['distance_to_roads'] = np.nan
    for p in range (len(results_landuse_polygons)):
        nearest=(point_tree.query_ball_point([results_landuse_polygons['centroid'].iloc[p].x, results_landuse_polygons['centroid'].iloc[p].y], 1))
        if (len(nearest)==0):
            nearest=(point_tree.query_ball_point([results_landuse_polygons['centroid'].iloc[p].x, results_landuse_polygons['centroid'].iloc[p].y], 2))
        if (len(nearest)==0):
            nearest=(point_tree.query_ball_point([results_landuse_polygons['centroid'].iloc[p].x, results_landuse_polygons['centroid'].iloc[p].y], 20))
        gdf_nearest_roads = geopandas.GeoDataFrame(columns=roads_gdf.columns)
        gdf_nearest_roads=roads_gdf.iloc[(nearest)]
        results_landuse_polygons['distance_to_roads'].iloc[p]= road_distance(results_landuse_polygons['centroid'].iloc[p], gdf_nearest_roads)
        price=70 #thousand euros per km
        results_landuse_polygons['road_cost']=results_landuse_polygons['distance_to_roads']*price
    return results_landuse_polygons

In [64]:
results_landuse_1=distance_to_roads(results_landuse_1)

In [65]:
results_landuse_2=distance_to_roads(results_landuse_2)

In [66]:
results_landuse_3=distance_to_roads(results_landuse_3)

In [67]:
results_landuse_4=distance_to_roads(results_landuse_4)

In [68]:
results_landuse_5=distance_to_roads(results_landuse_5)

In [69]:
results_landuse_6=distance_to_roads(results_landuse_6)

In [70]:
results_landuse_7=distance_to_roads(results_landuse_7)

In [71]:
results_landuse_8=distance_to_roads(results_landuse_8)

In [72]:
results_landuse_9=distance_to_roads(results_landuse_9)

In [73]:
results_landuse_10=distance_to_roads(results_landuse_10)

In [74]:
results_landuse_11=distance_to_roads(results_landuse_11)

In [75]:
"""
results_landuse_1.to_csv('results_landuse+costs_1.csv')
results_landuse_2.to_csv('results_landuse+costs_2.csv')
results_landuse_3.to_csv('results_landuse+costs_3.csv')
results_landuse_4.to_csv('results_landuse+costs_4.csv')
results_landuse_5.to_csv('results_landuse+costs_5.csv')
results_landuse_6.to_csv('results_landuse+costs_6.csv')
results_landuse_7.to_csv('results_landuse+costs_7.csv')
results_landuse_8.to_csv('results_landuse+costs_8.csv')
results_landuse_9.to_csv('results_landuse+costs_9.csv')
results_landuse_10.to_csv('results_landuse+costs_10.csv')
results_landuse_11.to_csv('results_landuse+costs_11.csv')
"""

## Map bidding zones


In [77]:
#join all separate files
results_landuse_polygons=results_landuse_1.append(results_landuse_2).append(results_landuse_3).append(results_landuse_4).append(results_landuse_5).append(results_landuse_6).append(results_landuse_7).append(results_landuse_8).append(results_landuse_9).append(results_landuse_10).append(results_landuse_11)

In [78]:
italy_gdf = geopandas.read_file('/Users/alexandrapopova/Downloads/Italy_Zones.shp')

In [79]:
sweden_gdf = geopandas.read_file('/Users/alexandrapopova/Downloads/Sweden_Zones.shp')

In [80]:
denmark_gdf = geopandas.read_file('/Users/alexandrapopova/Downloads/Denmark_Zones.shp')

In [81]:
italy_gdf = italy_gdf.dissolve(by='zone')

In [82]:
denmark_gdf = denmark_gdf.dissolve(by='zone')

In [83]:
italy_gdf=italy_gdf.reset_index()

In [84]:
denmark_gdf=denmark_gdf.reset_index()

In [85]:
results_landuse_zones=results_landuse_polygons.loc[(results_landuse_polygons['ISO3_CODE'] != 'ITA')&(results_landuse_polygons['ISO3_CODE'] != 'SWE')&(results_landuse_polygons['ISO3_CODE'] != 'DNK')]
results_landuse_italy=results_landuse_polygons.loc[(results_landuse_polygons['ISO3_CODE'] == 'ITA')]
results_landuse_sweden=results_landuse_polygons.loc[(results_landuse_polygons['ISO3_CODE'] == 'SWE')]
results_landuse_denmark=results_landuse_polygons.loc[(results_landuse_polygons['ISO3_CODE'] == 'DNK')]

In [86]:
#intersect with bidding zones
results_landuse_italy=geopandas.overlay(results_landuse_italy, italy_gdf, how='intersection')


/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_9298/3509756867.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  results_landuse_italy=geopandas.overlay(results_landuse_italy, italy_gdf, how='intersection')


In [87]:
#intersect with bidding zones
results_landuse_sweden=geopandas.overlay(results_landuse_sweden, sweden_gdf, how='intersection')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_9298/534446838.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  results_landuse_sweden=geopandas.overlay(results_landuse_sweden, sweden_gdf, how='intersection')


In [88]:
#intersect with bidding zones
results_landuse_denmark=geopandas.overlay(results_landuse_denmark, denmark_gdf, how='intersection')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_9298/95641466.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  results_landuse_denmark=geopandas.overlay(results_landuse_denmark, denmark_gdf, how='intersection')


In [89]:
#create zone code
results_landuse_sweden['zone']=results_landuse_sweden['ISO3_CODE']+'-'+results_landuse_sweden['zone']

In [90]:
results_landuse_zones['zone']=results_landuse_zones['ISO3_CODE']

In [91]:
#join all bidding zones back together
results_landuse_zones=results_landuse_zones.append(results_landuse_italy).append(results_landuse_sweden).append(results_landuse_denmark)

In [92]:
results_landuse_zones=results_landuse_zones.reset_index()

In [93]:
results_landuse_zones=results_landuse_zones.drop(columns=['FID', 'index'])

In [94]:
#filter out small polygons
results_landuse_zones=results_landuse_zones.loc[(results_landuse_zones['geometry'].area>0.0001)]

In [95]:
#results_landuse_zones.to_csv('results_landuse_cost_zones.csv')

## Append energy by hour

In [97]:
#at this point the dataset is split into all possible polygons, 
#but has only 1 line for each location - need to multiply it by seasons and hour

results_landuse_zones['hour']=1
results_landuse_zones['season']="ss"

In [98]:
results_landuse_zones['hour']=1
results_landuse_zones['season']="ss"
df_expanded=results_landuse_zones.copy()

for h in range(23):
    results_landuse_zones['hour']=h+2
    df_expanded=df_expanded.append(results_landuse_zones)

results_landuse_zones['hour']=1
results_landuse_zones['season']="aw"
df_expanded=df_expanded.append(results_landuse_zones)
for h in range(23):
    results_landuse_zones['hour']=h+2
    df_expanded=df_expanded.append(results_landuse_zones)

In [99]:
df_expanded=df_expanded.reset_index()

In [100]:
df_expanded=df_expanded.sort_values(by=['index', 'hour','season'])

In [101]:
df_expanded['coords-season-hour'] = df_expanded['coords']+'-'+ df_expanded['season']+'-'+ df_expanded['hour'].astype(str)

In [115]:
#append energy values by location, hour, zone
df_expanded['Vestas_Energy_Min_MWh'] = df_expanded['coords-season-hour'].map(aggr_df.set_index('coords-season-hour')['Vestas_Energy_Min_MWh'].to_dict())
df_expanded['Vestas_Energy_Max_MWh'] = df_expanded['coords-season-hour'].map(aggr_df.set_index('coords-season-hour')['Vestas_Energy_Max_MWh'].to_dict())
df_expanded['Nordex_Energy_Min_MWh'] = df_expanded['coords-season-hour'].map(aggr_df.set_index('coords-season-hour')['Nordex_Energy_Min_MWh'].to_dict())
df_expanded['Nordex_Energy_Max_MWh'] = df_expanded['coords-season-hour'].map(aggr_df.set_index('coords-season-hour')['Nordex_Energy_Max_MWh'].to_dict())
df_expanded['Enercon_Energy_Min_MWh'] = df_expanded['coords-season-hour'].map(aggr_df.set_index('coords-season-hour')['Enercon_Energy_Min_MWh'].to_dict())
df_expanded['Enercon_Energy_Max_MWh'] = df_expanded['coords-season-hour'].map(aggr_df.set_index('coords-season-hour')['Enercon_Energy_Max_MWh'].to_dict())
df_expanded['SolarPV_Energy_Min_MWh'] = df_expanded['coords-season-hour'].map(aggr_df.set_index('coords-season-hour')['SolarPV_Energy_Min_MWh'].to_dict())
df_expanded['SolarPV_Energy_Max_MWh'] = df_expanded['coords-season-hour'].map(aggr_df.set_index('coords-season-hour')['SolarPV_Energy_Max_MWh'].to_dict())

## Append costs and prices

In [116]:
OM_cost_df = pd.read_csv('OM_cost.csv')

In [117]:
#append O&M costs based on the country code
df_expanded['OM cost, EUR/kWh'] = df_expanded['ISO3_CODE'].map(OM_cost_df.set_index('Country')['OM cost, EUR/kWh'].to_dict())

In [118]:
price_df = pd.read_csv('Price by country.csv')

In [119]:
#create identifier for lookups
df_expanded['Zone-Season-Hour']=df_expanded['zone']+"-"+df_expanded['season']+"-"+df_expanded['hour'].astype(str)

In [120]:
#append prices based on the zone/season/hour identifier
df_expanded['Price, EUR/MWh']= df_expanded['Zone-Season-Hour'].map(price_df.set_index('Zone - Season - Hour')['Price'].to_dict())




In [121]:
wacc_df = pd.read_csv('WACC by country.csv')

In [122]:
#append WACC based on the country code
df_expanded['WACC, %']= df_expanded['ISO3_CODE'].map(wacc_df.set_index('Country')['WACC, %'].to_dict())

In [366]:
#all locations now have all technical and economic metrics attached
df_expanded.to_csv('df_expanded_preNPV.csv')

In [5]:
#df_expanded=pd.read_csv('df_expanded_preNPV.csv')

/opt/anaconda3/envs/geo_env2/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## NPV calculation

In [6]:
#create identifier of location alternative
df_expanded = df_expanded.assign(loc_id=(df_expanded['geometry']).astype('category').cat.codes)

In [7]:
df_expanded =df_expanded.sort_values(by=['loc_id', 'season','hour'])

In [8]:
#Revenue = Price x Energy
def Recurring_Revenue (season, energy, price):
    recurring_revenue=np.where((season=='ss'), energy*(price)*184, energy*(price)*181)
    return recurring_revenue

In [9]:
#O&M Costs = O&M per kWh x Energy
def Recurring_Cost (season, energy, OM):
    recurring_cost=np.where((season=='ss'), energy*(OM*1000)*184, energy*(OM*1000)*181)
    return recurring_cost

In [10]:
#Find mean energy values
df_expanded['Vestas_Energy_Mean_MWh']=df_expanded['Vestas_Energy_Min_MWh']+(df_expanded['Vestas_Energy_Max_MWh']-df_expanded['Vestas_Energy_Min_MWh'])/2
df_expanded['Nordex_Energy_Mean_MWh']=df_expanded['Nordex_Energy_Min_MWh']+(df_expanded['Nordex_Energy_Max_MWh']-df_expanded['Nordex_Energy_Min_MWh'])/2
df_expanded['Enercon_Energy_Mean_MWh']=df_expanded['Enercon_Energy_Min_MWh']+(df_expanded['Enercon_Energy_Max_MWh']-df_expanded['Enercon_Energy_Min_MWh'])/2
df_expanded['SolarPV_Energy_Mean_MWh']=df_expanded['SolarPV_Energy_Min_MWh']+(df_expanded['SolarPV_Energy_Max_MWh']-df_expanded['SolarPV_Energy_Min_MWh'])/2


In [11]:
#Calculate revenues and O&M costs
df_expanded['Vestas_Recurring_Revenue']=Recurring_Revenue(df_expanded['season'],df_expanded['Vestas_Energy_Mean_MWh'],df_expanded['Price, EUR/MWh'])
df_expanded['Nordex_Recurring_Revenue']=Recurring_Revenue(df_expanded['season'],df_expanded['Nordex_Energy_Mean_MWh'],df_expanded['Price, EUR/MWh'])
df_expanded['Enercon_Recurring_Revenue']=Recurring_Revenue(df_expanded['season'],df_expanded['Enercon_Energy_Mean_MWh'],df_expanded['Price, EUR/MWh'])
df_expanded['SolarPV_Recurring_Revenue']=Recurring_Revenue(df_expanded['season'],df_expanded['SolarPV_Energy_Mean_MWh'],df_expanded['Price, EUR/MWh'])

df_expanded['Vestas_Recurring_Cost']=Recurring_Cost(df_expanded['season'],df_expanded['Vestas_Energy_Mean_MWh'],df_expanded['OM cost, EUR/kWh'])
df_expanded['Nordex_Recurring_Cost']=Recurring_Cost(df_expanded['season'],df_expanded['Nordex_Energy_Mean_MWh'],df_expanded['OM cost, EUR/kWh'])
df_expanded['Enercon_Recurring_Cost']=Recurring_Cost(df_expanded['season'],df_expanded['Enercon_Energy_Mean_MWh'],df_expanded['OM cost, EUR/kWh'])
df_expanded['SolarPV_Recurring_Cost']=Recurring_Cost(df_expanded['season'],df_expanded['SolarPV_Energy_Mean_MWh'],df_expanded['OM cost, EUR/kWh'])

In [13]:
#Aggregate into annual values
df_expanded_sum=df_expanded.groupby(['loc_id']).agg({'Vestas_Recurring_Revenue': ['sum'],
                                                        'Nordex_Recurring_Revenue': ['sum'],
                                                        'Enercon_Recurring_Revenue': ['sum'],
                                                        'SolarPV_Recurring_Revenue': ['sum'],
                                                        'Vestas_Recurring_Cost': ['sum'],
                                                        'Nordex_Recurring_Cost': ['sum'],
                                                        'Enercon_Recurring_Cost': ['sum'],
                                                        'SolarPV_Recurring_Cost': ['sum'],
                                                        'Vestas_Energy_Mean_MWh':['sum'],
                                                        'Nordex_Energy_Mean_MWh':['sum'],
                                                        'Enercon_Energy_Mean_MWh':['sum'],
                                                        'SolarPV_Energy_Mean_MWh':['sum']})


In [14]:
df_expanded_sum=df_expanded_sum.reset_index()

In [15]:
df_expanded_sum.columns=df_expanded_sum.columns.droplevel(-1)

In [16]:
#Daily energy sum needs to be divided by 2 to find average between the 2 seasons
df_expanded_sum['Vestas_Energy_Mean_MWh']=df_expanded_sum['Vestas_Energy_Mean_MWh']/2
df_expanded_sum['Nordex_Energy_Mean_MWh']=df_expanded_sum['Nordex_Energy_Mean_MWh']/2
df_expanded_sum['Enercon_Energy_Mean_MWh']=df_expanded_sum['Enercon_Energy_Mean_MWh']/2
df_expanded_sum['SolarPV_Energy_Mean_MWh']=df_expanded_sum['SolarPV_Energy_Mean_MWh']/2


In [17]:
#Append information fields
df_expanded_sum['Country']=df_expanded_sum['loc_id'].map(df_expanded.set_index('loc_id')['ISO3_CODE'].to_dict())
df_expanded_sum['Zone']=df_expanded_sum['loc_id'].map(df_expanded.set_index('loc_id')['zone'].to_dict())
df_expanded_sum['Line_Length_km']=df_expanded_sum['loc_id'].map(df_expanded.set_index('loc_id')['lines_length'].to_dict())
df_expanded_sum['Lines_Cost_kEUR']=df_expanded_sum['loc_id'].map(df_expanded.set_index('loc_id')['lines_cost'].to_dict())
df_expanded_sum['Distance_to_roads_km']=df_expanded_sum['loc_id'].map(df_expanded.set_index('loc_id')['distance_to_roads'].to_dict())
df_expanded_sum['Road_Cost_kEUR']=df_expanded_sum['loc_id'].map(df_expanded.set_index('loc_id')['road_cost'].to_dict())
df_expanded_sum['WACC, %']=df_expanded_sum['loc_id'].map(df_expanded.set_index('loc_id')['WACC, %'].to_dict())
df_expanded_sum['geometry']=df_expanded_sum['loc_id'].map(df_expanded.set_index('loc_id')['geometry'].to_dict())


In [18]:
#Calculate present value of recurring profits
df_expanded_sum['Vestas Profit PV, kEUR']=((df_expanded_sum['Vestas_Recurring_Revenue']-df_expanded_sum['Vestas_Recurring_Cost'])/1000)*(1-(1+df_expanded_sum['WACC, %']/100)**(-30))/(df_expanded_sum['WACC, %']/100)
df_expanded_sum['Nordex Profit PV, kEUR']=((df_expanded_sum['Nordex_Recurring_Revenue']-df_expanded_sum['Nordex_Recurring_Cost'])/1000)*(1-(1+df_expanded_sum['WACC, %']/100)**(-30))/(df_expanded_sum['WACC, %']/100)
df_expanded_sum['Enercon Profit PV, kEUR']=((df_expanded_sum['Enercon_Recurring_Revenue']-df_expanded_sum['Enercon_Recurring_Cost'])/1000)*(1-(1+df_expanded_sum['WACC, %']/100)**(-30))/(df_expanded_sum['WACC, %']/100)
df_expanded_sum['SolarPV Profit PV, kEUR']=((df_expanded_sum['SolarPV_Recurring_Revenue']-df_expanded_sum['SolarPV_Recurring_Cost'])/1000)*(1-(1+df_expanded_sum['WACC, %']/100)**(-30))/(df_expanded_sum['WACC, %']/100)

In [19]:
#Calculate investment costs
df_expanded_sum['Vestas Investment Cost, kEUR']=1000+8.5*3.5+100+df_expanded_sum['Lines_Cost_kEUR']+df_expanded_sum['Road_Cost_kEUR']
df_expanded_sum['Nordex Investment Cost, kEUR']=500+8.5*2.5+100+df_expanded_sum['Lines_Cost_kEUR']+df_expanded_sum['Road_Cost_kEUR']
df_expanded_sum['Enercon Investment Cost, kEUR']=200+8.5*0.8+100+df_expanded_sum['Lines_Cost_kEUR']+df_expanded_sum['Road_Cost_kEUR']
df_expanded_sum['SolarPV Investment Cost, kEUR']=750+8.5+100+df_expanded_sum['Lines_Cost_kEUR']+df_expanded_sum['Road_Cost_kEUR']

In [20]:
#Calculate NPV
df_expanded_sum['Vestas NPV, kEUR']=df_expanded_sum['Vestas Profit PV, kEUR']-df_expanded_sum['Vestas Investment Cost, kEUR']
df_expanded_sum['Nordex NPV, kEUR']=df_expanded_sum['Nordex Profit PV, kEUR']-df_expanded_sum['Nordex Investment Cost, kEUR']
df_expanded_sum['Enercon NPV, kEUR']=df_expanded_sum['Enercon Profit PV, kEUR']-df_expanded_sum['Enercon Investment Cost, kEUR']
df_expanded_sum['SolarPV NPV, kEUR']=df_expanded_sum['SolarPV Profit PV, kEUR']-df_expanded_sum['SolarPV Investment Cost, kEUR']


In [21]:
def calc_IRR(cf):
    cashflow=np.empty(31)
    cashflow.fill(cf[1])
    cashflow[0]=cf[0]
    irr = round(npf.irr(cashflow),4)
    return irr

In [22]:
#Calculate Cash Flow for IRR definition
df_expanded_sum['Vestas CF']=list(zip(-df_expanded_sum['Vestas Investment Cost, kEUR'],(df_expanded_sum['Vestas_Recurring_Revenue']-df_expanded_sum['Vestas_Recurring_Cost'])/1000))
df_expanded_sum['Nordex CF']=list(zip(-df_expanded_sum['Nordex Investment Cost, kEUR'],(df_expanded_sum['Nordex_Recurring_Revenue']-df_expanded_sum['Nordex_Recurring_Cost'])/1000))
df_expanded_sum['Enercon CF']=list(zip(-df_expanded_sum['Enercon Investment Cost, kEUR'],(df_expanded_sum['Enercon_Recurring_Revenue']-df_expanded_sum['Enercon_Recurring_Cost'])/1000))
df_expanded_sum['SolarPV CF']=list(zip(-df_expanded_sum['SolarPV Investment Cost, kEUR'],(df_expanded_sum['SolarPV_Recurring_Revenue']-df_expanded_sum['SolarPV_Recurring_Cost'])/1000))





In [23]:
#Calculate IRR
df_expanded_sum['Vestas IRR']=df_expanded_sum['Vestas CF'].apply(lambda x: calc_IRR(x))
df_expanded_sum['Nordex IRR']=df_expanded_sum['Nordex CF'].apply(lambda x: calc_IRR(x))
df_expanded_sum['Enercon IRR']=df_expanded_sum['Enercon CF'].apply(lambda x: calc_IRR(x))
df_expanded_sum['SolarPV IRR']=df_expanded_sum['SolarPV CF'].apply(lambda x: calc_IRR(x))
                                       

In [24]:
#Calculate IRR
df_expanded_sum['Vestas Payback']=round(df_expanded_sum['Vestas Investment Cost, kEUR']*1000/(df_expanded_sum['Vestas_Recurring_Revenue']-df_expanded_sum['Vestas_Recurring_Cost']),2)
df_expanded_sum['Nordex Payback']=round(df_expanded_sum['Nordex Investment Cost, kEUR']*1000/(df_expanded_sum['Nordex_Recurring_Revenue']-df_expanded_sum['Nordex_Recurring_Cost']),2)
df_expanded_sum['Enercon Payback']=round(df_expanded_sum['Enercon Investment Cost, kEUR']*1000/(df_expanded_sum['Enercon_Recurring_Revenue']-df_expanded_sum['Enercon_Recurring_Cost']),2)
df_expanded_sum['SolarPV Payback']=round(df_expanded_sum['SolarPV Investment Cost, kEUR']*1000/(df_expanded_sum['SolarPV_Recurring_Revenue']-df_expanded_sum['SolarPV_Recurring_Cost']),2)

In [287]:
df_expanded_sum=df_expanded_sum.drop(columns=['Vestas CF','Nordex CF','Enercon CF','SolarPV CF'])

In [358]:
NPV_gdf = geopandas.GeoDataFrame(df_expanded_sum, geometry='geometry')

In [ ]:
NPV_gdf.to_file("NPV_map.shp")

In [289]:
#EPSG:4326 needed for area caclulation
NPV_gdf=NPV_gdf.set_crs("EPSG:4326")
NPV_gdf_epsg3857=NPV_gdf.to_crs({'init': 'epsg:3857'})
NPV_gdf_epsg3857['area']=NPV_gdf_epsg3857['geometry'].area/10**6
NPV_gdf_epsg3857=NPV_gdf_epsg3857[['loc_id','area']]
NPV_gdf_epsg3857.to_csv("NPV_epsg3857.csv")

In [2]:
#df_expanded_sum=geopandas.read_file("NPV_map.shp")

In [27]:
#calculate average NPV between 4 technologies
df_expanded_sum['Average_NPV']=0
for i in range(len(df_expanded_sum)):
    if df_expanded_sum['SolarPV_Energy_Mean_MWh'][i]==0:
        df_expanded_sum['Average_NPV'][i]=(df_expanded_sum['Vestas NPV, kEUR'][i]+df_expanded_sum['Nordex NPV, kEUR'][i]+df_expanded_sum['Enercon NPV, kEUR'][i])/3
    else:
        df_expanded_sum['Average_NPV'][i]=(df_expanded_sum['Vestas NPV, kEUR'][i]+df_expanded_sum['Nordex NPV, kEUR'][i]+df_expanded_sum['Enercon NPV, kEUR'][i]+df_expanded_sum['SolarPV NPV, kEUR'][i])/4   

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_2341/871543147.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_expanded_sum['Average_NPV'][i]=(df_expanded_sum['Vestas NPV, kEUR'][i]+df_expanded_sum['Nordex NPV, kEUR'][i]+df_expanded_sum['Enercon NPV, kEUR'][i])/3
/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_2341/871543147.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_expanded_sum['Average_NPV'][i]=(df_expanded_sum['Vestas NPV, kEUR'][i]+df_expanded_sum['Nordex NPV, kEUR'][i]+df_expanded_sum['Enercon NPV, kEUR'][i]+df_expanded_sum['SolarPV NPV, kEUR'][i])/4


## Simplify to polygons - for web-tool display

In [57]:
df_expanded_sum['geometry']=df_expanded_sum['geometry'].apply(lambda x: wkt.loads(x))

In [60]:
df_expanded_sum['centroid']=df_expanded_sum['geometry'].apply(lambda x: x.centroid)

In [61]:
#assign each location polygon to a cell
for i in range (len(df_expanded_sum)):
    point=df_expanded_sum['centroid'][i]
    for j in range (len(results)):
        if point.within(results['geometry'][j]):
            df_expanded_sum['polygon'][i]=str(results['geometry'][j])
        else:
            continue

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_2341/558145808.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_expanded_sum['polygon'][i]=str(results['geometry'][j])


In [64]:
#find total cell area - for average weighted NPV
df_expanded_sum['polygon_area']=df_expanded_sum['polygon'].apply(lambda x: x.area)

In [66]:
#find the location area - for average weighted NPV
df_expanded_sum['accessible_area']=df_expanded_sum['geometry'].apply(lambda x: x.area)

In [67]:
#weigh the NPV Average for 4 equipment models
df_expanded_sum['weighted_NPV_Average']=df_expanded_sum['accessible_area']*df_expanded_sum['Average_NPV']

In [68]:
#weigh the NPV Average for Vestas
df_expanded_sum['weighted_NPV_Vestas']=df_expanded_sum['accessible_area']*df_expanded_sum['Vestas NPV, kEUR']

In [69]:
#aggregate by cell
df_expanded_sum['polygon']=df_expanded_sum['polygon'].astype(str)
df_expanded_polygons=df_expanded_sum.groupby(['polygon'] ).agg({'weighted_NPV_Average': ['sum'],'weighted_NPV_Vestas': ['sum'],'polygon_area':['mean']})
df_expanded_polygons=df_expanded_polygons.reset_index()
df_expanded_polygons.columns=df_expanded_polygons.columns.droplevel(-1)


In [73]:
#find average weighted NPV
df_expanded_polygons['average_weighted_NPV_all']=df_expanded_polygons['weighted_NPV_Average']/df_expanded_polygons['polygon_area']
df_expanded_polygons['average_weighted_NPV_Vestas']=df_expanded_polygons['weighted_NPV_Vestas']/df_expanded_polygons['polygon_area']

In [76]:
df_expanded_polygons['polygon']=df_expanded_polygons['polygon'].apply(lambda x: shapely.wkt.loads(x))

In [77]:
NPV_polygons_gdf = geopandas.GeoDataFrame(df_expanded_polygons, geometry='polygon')

In [78]:
NPV_polygons_gdf.to_file("NPV_polygons_map.shp")

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_2341/3768200808.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  NPV_polygons_gdf.to_file("NPV_polygons_map.shp")


## Cost optimisation model

In [292]:
#create array with location attribution to zones
zone_list=df_expanded_sum['Zone'].drop_duplicates().sort_values().reset_index()['Zone'].tolist()
loc_zones = pd.DataFrame(0, index=range(0,len(df_expanded_sum)),columns=zone_list)

for column in loc_zones:
    for i in range(0,len(df_expanded_sum)):
        if (df_expanded_sum.Zone[i]==column):
            loc_zones[column][i]=1
        else:
            continue

loc_zones_array=loc_zones.to_numpy()

In [385]:
from numpy import savetxt
from numpy import loadtxt
savetxt('loc_zones_array.csv', loc_zones_array, delimiter=',')

In [296]:
#mask array for zone selection
selected_zone='FRA'
df_expanded_sum['mask']=np.where((df_expanded_sum['Zone']==selected_zone), 1, 0)
mask_array=df_expanded_sum['mask'].to_numpy()

In [295]:
#create energy array
energy_df = pd.DataFrame(0, index=range(0,len(df_expanded_sum)),columns=['Vestas', 'Nordex', 'Enercon', 'SolarPV'])

for i in range(0,len(df_expanded_sum)):
    energy_df['Vestas'][i]=np.array(df_expanded[df_expanded['loc_id']==i]['Vestas_Energy_Max_MWh']).astype(object)
    energy_df['Nordex'][i]=np.array(df_expanded[df_expanded['loc_id']==i]['Nordex_Energy_Max_MWh']).astype(object)
    energy_df['Enercon'][i]=np.array(df_expanded[df_expanded['loc_id']==i]['Enercon_Energy_Max_MWh']).astype(object)
    energy_df['SolarPV'][i]=np.array(df_expanded[df_expanded['loc_id']==i]['SolarPV_Energy_Max_MWh']).astype(object)
    
energy_array=energy_df.to_numpy()

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/2388968071.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_df['Nordex'][i]=np.array(df_expanded[df_expanded['loc_id']==i]['Nordex_Energy_Max_MWh']).astype(object)
/opt/anaconda3/envs/geo_env2/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/2388968071.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

In [ ]:
#aggregate by season
energy_array_seasonal = np.zeros((len(df_expanded_sum),4,2)) 
for i in range(len(df_expanded_sum)):
    for e in range(4):
        energy_array_seasonal[i][e][0]=sum(energy_array[i][e][0:24])
        energy_array_seasonal[i][e][1]=sum(energy_array[i][e][24:48])

In [ ]:
#split into 2 arrays - for loading into the web tool
energy_array_aw = np.zeros((len(df_expanded_sum),4)) 
energy_array_ss = np.zeros((len(df_expanded_sum),4)) 

for i in range(len(df_expanded_sum)):
    for e in range(4):
        energy_array_aw[i][e]=sum(energy_array[i][e][0:24])
        energy_array_ss[i][e]=sum(energy_array[i][e][24:48])
        
savetxt('energy_array_aw.csv', energy_array_aw, delimiter=',')
savetxt('energy_array_ss.csv', energy_array_ss, delimiter=',')

In [297]:
#create demand array by season, for constraining energy production
demand_df=pd.read_csv('Net_Demand_Seasonal.csv')
demand_const= pd.DataFrame(0, index=range(0,1),columns=zone_list)
for column in demand_const:
    demand_const[column][0]=(np.array(demand_df[column])).astype(object)
demand_const_array=demand_const.to_numpy()[0]

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/1082714821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demand_const[column][0]=(np.array(demand_df[column])).astype(object)


In [400]:
#split into 2 arrays - for loading into the web tool
demand_array_aw = np.zeros((len(demand_const_array))) 
demand_array_ss = np.zeros((len(demand_const_array))) 
for i in range(len(demand_const_array)):
    demand_array_aw[i]=demand_const_array[i][0]
    demand_array_ss[i]=demand_const_array[i][1]
savetxt('demand_array_aw.csv', demand_array_aw, delimiter=',')
savetxt('demand_array_ss.csv', demand_array_ss, delimiter=',')

In [ ]:
#create array of area constraints
loc_area=NPV_gdf_epsg3857['area'].to_numpy()
savetxt('loc_area.csv', loc_area, delimiter=',')

In [299]:
#model variable coefficients
v_cost=df_expanded_sum['Vestas Investment Cost, kEUR'].to_numpy()
n_cost=df_expanded_sum['Nordex Investment Cost, kEUR'].to_numpy()
e_cost=df_expanded_sum['Enercon Investment Cost, kEUR'].to_numpy()
s_cost=df_expanded_sum['SolarPV Investment Cost, kEUR'].to_numpy()

v_NPV=df_expanded_sum['Vestas NPV, kEUR'].to_numpy()
n_NPV=df_expanded_sum['Nordex NPV, kEUR'].to_numpy()
e_NPV=df_expanded_sum['Enercon NPV, kEUR'].to_numpy()
s_NPV=df_expanded_sum['SolarPV NPV, kEUR'].to_numpy()


In [303]:
#cost minimisation model - seasonal energy constraint
import gurobipy as gp
from gurobipy import GRB
I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=100000
m=gp.Model()
x = [[m.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m.addConstr(((x[i][0]+x[i][1]+x[i][2])*0.8+x[i][3]*0.02) <= loc_area[i])
for z in Z:
    for s in S:
        m.addConstr(gp.quicksum((energy_array_seasonal[i][0][s]*x[i][0]+energy_array_seasonal[i][1][s]*x[i][1]+energy_array_seasonal[i][2][s]*x[i][2]+energy_array_seasonal[i][3][s]*x[i][3])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m.addConstr(gp.quicksum(v_cost[i]*x[i][0] + n_cost[i]*x[i][1] + e_cost[i]*x[i][2] + s_cost[i]*x[i][3] for i in I)<=cost_max)        
m.setObjective(gp.quicksum(v_NPV[i] * x[i][0]*mask_array[i]+n_NPV[i]*x[i][1]*mask_array[i]+e_NPV[i]*x[i][2]*mask_array[i]+s_NPV[i]*x[i][3]*mask_array[i] for i in I), GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 1998218 nonzeros
Model fingerprint: 0x51ef5253
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+05]
  Objective range  [1e-02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+05]
Found heuristic solution: objective 84732.790414
Presolve removed 120557 rows and 487705 columns
Presolve time: 0.83s
Presolved: 5986 rows, 18167 columns, 30886 nonzeros
Variable types: 0 continuous, 18167 integer (0 binary)

Root relaxation: objective 5.405622e+05, 6063 iterations, 2.41 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 540562.155    0    1 84732.7904 540562.155   538%     -    4s
H    0     0                

In [305]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_Initial.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/112223275.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  solution_gdf.to_file('solution_Initial.shp')


In [325]:
solution_Initial = geopandas.read_file('solution_Initial.shp')
solution_Initial.to_csv('solution_Initial.csv')

In [182]:
#cost minimisation model - hourly energy constraint
import gurobipy as gp
from gurobipy import GRB
I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
H=range(48)
cost_max=100000
m=gp.Model()
x = [[m.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m.addConstr(((x[i][0]+x[i][1]+x[i][2])*0.8+x[i][3]*0.02) <= loc_area[i])
for z in Z:
    for h in H:
        m.addConstr(gp.quicksum((energy_array[i][0][h]*x[i][0]+energy_array[i][1][h]*x[i][1]+energy_array[i][2][h]*x[i][2]+energy_array[i][3][h]*x[i][3])*loc_zones_array[i][z] for i in I) <=demand_const_array_h[z][h])      

m.addConstr(gp.quicksum(v_cost[i]*x[i][0] + n_cost[i]*x[i][1] + e_cost[i]*x[i][2] + s_cost[i]*x[i][3] for i in I)<=cost_max)        
m.setObjective(gp.quicksum(v_NPV[i] * x[i][0]*mask_array[i]+n_NPV[i]*x[i][1]*mask_array[i]+e_NPV[i]*x[i][2]*mask_array[i]+s_NPV[i]*x[i][3]*mask_array[i] for i in I), GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 128245 rows, 505872 columns and 22489829 nonzeros
Model fingerprint: 0x9f915a33
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-07, 2e+05]
  Objective range  [1e-01, 4e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 160527.92925
Presolve removed 125937 rows and 498692 columns (presolve time = 5s) ...
Presolve removed 126198 rows and 498692 columns
Presolve time: 5.30s
Presolved: 2047 rows, 7180 columns, 11272 nonzeros
Variable types: 0 continuous, 7180 integer (0 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4673829e+08   2.532237e+06   0.000000e+00      

In [306]:
#cost minimisation model - no country constraint
I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=100000
m1=gp.Model()
x = [[m1.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m1.addConstr(((x[i][0]+x[i][1]+x[i][2])*0.8+x[i][3]*0.02) <= loc_area[i])
for z in Z:
    for s in S:
        m1.addConstr(gp.quicksum((energy_array_seasonal[i][0][s]*x[i][0]+energy_array_seasonal[i][1][s]*x[i][1]+energy_array_seasonal[i][2][s]*x[i][2]+energy_array_seasonal[i][3][s]*x[i][3])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m1.addConstr(gp.quicksum(v_cost[i]*x[i][0] + n_cost[i]*x[i][1] + e_cost[i]*x[i][2] + s_cost[i]*x[i][3] for i in I)<=cost_max)        
m1.setObjective(gp.quicksum(v_NPV[i] * x[i][0]+n_NPV[i]*x[i][1]+e_NPV[i]*x[i][2]+s_NPV[i]*x[i][3] for i in I), GRB.MAXIMIZE)
m1.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 1998218 nonzeros
Model fingerprint: 0xc7a28eeb
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+05]
  Objective range  [1e-02, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+05]
Found heuristic solution: objective 45593.144900
Presolve removed 106744 rows and 439786 columns
Presolve time: 1.35s
Presolved: 19799 rows, 66086 columns, 108384 nonzeros
Variable types: 0 continuous, 66086 integer (16 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective 8.146577e+05, 1 iterations, 0.15 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [307]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m1.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m1.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_NoCountry.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/1653935046.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  solution_gdf.to_file('solution_NoCountry.shp')


In [326]:
solution_NoCountry = geopandas.read_file('solution_NoCountry.shp')
solution_NoCountry.to_csv('solution_NoCountry.csv')

In [308]:
#cost minimisation model - different costs

I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=10000
m2=gp.Model()
x = [[m2.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m2.addConstr(((x[i][0]+x[i][1]+x[i][2])*0.8+x[i][3]*0.02) <= loc_area[i])
for z in Z:
    for s in S:
        m2.addConstr(gp.quicksum((energy_array_seasonal[i][0][s]*x[i][0]+energy_array_seasonal[i][1][s]*x[i][1]+energy_array_seasonal[i][2][s]*x[i][2]+energy_array_seasonal[i][3][s]*x[i][3])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m2.addConstr(gp.quicksum(v_cost[i]*x[i][0] + n_cost[i]*x[i][1] + e_cost[i]*x[i][2] + s_cost[i]*x[i][3] for i in I)<=cost_max)        
m2.setObjective(gp.quicksum(v_NPV[i] * x[i][0]*mask_array[i]+n_NPV[i]*x[i][1]*mask_array[i]+e_NPV[i]*x[i][2]*mask_array[i]+s_NPV[i]*x[i][3]*mask_array[i] for i in I), GRB.MAXIMIZE)
m2.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 1998218 nonzeros
Model fingerprint: 0xd2964356
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+05]
  Objective range  [1e-02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+05]
Found heuristic solution: objective 11589.455243
Presolve removed 123186 rows and 487705 columns
Presolve time: 0.68s
Presolved: 3357 rows, 18167 columns, 25458 nonzeros
Variable types: 0 continuous, 18167 integer (88 binary)

Root relaxation: objective 5.381759e+04, 3724 iterations, 0.89 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 53817.5856    0    1 11589.4552 53817.5856   364%     -    1s
H    0     0               

In [309]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m2.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m2.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_10K.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/1716620687.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  solution_gdf.to_file('solution_10K.shp')


In [327]:
solution_10K = geopandas.read_file('solution_10K.shp')
solution_10K.to_csv('solution_10K.csv')

In [310]:
#cost minimisation model - different costs

I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=50000
m4=gp.Model()
x = [[m4.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m4.addConstr(((x[i][0]+x[i][1]+x[i][2])*0.8+x[i][3]*0.02) <= loc_area[i])
for z in Z:
    for s in S:
        m4.addConstr(gp.quicksum((energy_array_seasonal[i][0][s]*x[i][0]+energy_array_seasonal[i][1][s]*x[i][1]+energy_array_seasonal[i][2][s]*x[i][2]+energy_array_seasonal[i][3][s]*x[i][3])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m4.addConstr(gp.quicksum(v_cost[i]*x[i][0] + n_cost[i]*x[i][1] + e_cost[i]*x[i][2] + s_cost[i]*x[i][3] for i in I)<=cost_max)        
m4.setObjective(gp.quicksum(v_NPV[i] * x[i][0]*mask_array[i]+n_NPV[i]*x[i][1]*mask_array[i]+e_NPV[i]*x[i][2]*mask_array[i]+s_NPV[i]*x[i][3]*mask_array[i] for i in I), GRB.MAXIMIZE)
m4.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 1998218 nonzeros
Model fingerprint: 0x6edc62ba
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+05]
  Objective range  [1e-02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+05]
Found heuristic solution: objective 58174.143653
Presolve removed 120938 rows and 487705 columns
Presolve time: 0.63s
Presolved: 5605 rows, 18167 columns, 30095 nonzeros
Variable types: 0 continuous, 18167 integer (0 binary)

Root relaxation: objective 2.702811e+05, 5710 iterations, 1.71 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 270281.077    0    1 58174.1437 270281.077   365%     -    2s
H    0     0                

In [311]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m4.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m4.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_50K.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/307477105.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  solution_gdf.to_file('solution_50K.shp')


In [328]:
solution_50K = geopandas.read_file('solution_50K.shp')
solution_50K.to_csv('solution_50K.csv')

In [312]:
#cost minimisation model - different costs

I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=200000
m5=gp.Model()
x = [[m5.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m5.addConstr(((x[i][0]+x[i][1]+x[i][2])*0.8+x[i][3]*0.02) <= loc_area[i])
for z in Z:
    for s in S:
        m5.addConstr(gp.quicksum((energy_array_seasonal[i][0][s]*x[i][0]+energy_array_seasonal[i][1][s]*x[i][1]+energy_array_seasonal[i][2][s]*x[i][2]+energy_array_seasonal[i][3][s]*x[i][3])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m5.addConstr(gp.quicksum(v_cost[i]*x[i][0] + n_cost[i]*x[i][1] + e_cost[i]*x[i][2] + s_cost[i]*x[i][3] for i in I)<=cost_max)        
m5.setObjective(gp.quicksum(v_NPV[i] * x[i][0]*mask_array[i]+n_NPV[i]*x[i][1]*mask_array[i]+e_NPV[i]*x[i][2]*mask_array[i]+s_NPV[i]*x[i][3]*mask_array[i] for i in I), GRB.MAXIMIZE)
m5.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 1998218 nonzeros
Model fingerprint: 0x6b3e1d69
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+05]
  Objective range  [1e-02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+05]
Found heuristic solution: objective 99478.949107
Presolve removed 120331 rows and 487705 columns
Presolve time: 0.65s
Presolved: 6212 rows, 18167 columns, 31358 nonzeros
Variable types: 0 continuous, 18167 integer (0 binary)

Root relaxation: objective 1.081522e+06, 6261 iterations, 1.90 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1081522.03    0    1 99478.9491 1081522.03   987%     -    2s
H    0     0                

In [313]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m5.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m5.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_200K.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/1856312979.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  solution_gdf.to_file('solution_200K.shp')


In [329]:
solution_200K = geopandas.read_file('solution_200K.shp')
solution_200K.to_csv('solution_200K.csv')

In [314]:
#cost minimisation model - different costs

I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=500000
m6=gp.Model()
x = [[m6.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m6.addConstr(((x[i][0]+x[i][1]+x[i][2])*0.8+x[i][3]*0.02) <= loc_area[i])
for z in Z:
    for s in S:
        m6.addConstr(gp.quicksum((energy_array_seasonal[i][0][s]*x[i][0]+energy_array_seasonal[i][1][s]*x[i][1]+energy_array_seasonal[i][2][s]*x[i][2]+energy_array_seasonal[i][3][s]*x[i][3])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m6.addConstr(gp.quicksum(v_cost[i]*x[i][0] + n_cost[i]*x[i][1] + e_cost[i]*x[i][2] + s_cost[i]*x[i][3] for i in I)<=cost_max)        
m6.setObjective(gp.quicksum(v_NPV[i] * x[i][0]*mask_array[i]+n_NPV[i]*x[i][1]*mask_array[i]+e_NPV[i]*x[i][2]*mask_array[i]+s_NPV[i]*x[i][3]*mask_array[i] for i in I), GRB.MAXIMIZE)
m6.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 1998218 nonzeros
Model fingerprint: 0xf44902bb
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+05]
  Objective range  [1e-02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+05]
Found heuristic solution: objective 137642.50145
Presolve removed 120126 rows and 487705 columns
Presolve time: 0.73s
Presolved: 6417 rows, 18167 columns, 49956 nonzeros
Variable types: 0 continuous, 18167 integer (0 binary)

Root relaxation: objective 1.404570e+06, 6458 iterations, 2.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1404569.69    0    1 137642.501 1404569.69   920%     -    3s
H    0     0                

In [315]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m6.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m6.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_500k.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/1073804603.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  solution_gdf.to_file('solution_500k.shp')


In [330]:
solution_500K = geopandas.read_file('solution_500K.shp')
solution_500K.to_csv('solution_500K.csv')

In [316]:
#cost minimisation model - different costs

I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=1000000
m7=gp.Model()
x = [[m7.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m7.addConstr(((x[i][0]+x[i][1]+x[i][2])*0.8+x[i][3]*0.02) <= loc_area[i])
for z in Z:
    for s in S:
        m7.addConstr(gp.quicksum((energy_array_seasonal[i][0][s]*x[i][0]+energy_array_seasonal[i][1][s]*x[i][1]+energy_array_seasonal[i][2][s]*x[i][2]+energy_array_seasonal[i][3][s]*x[i][3])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m7.addConstr(gp.quicksum(v_cost[i]*x[i][0] + n_cost[i]*x[i][1] + e_cost[i]*x[i][2] + s_cost[i]*x[i][3] for i in I)<=cost_max)        
m7.setObjective(gp.quicksum(v_NPV[i] * x[i][0]*mask_array[i]+n_NPV[i]*x[i][1]*mask_array[i]+e_NPV[i]*x[i][2]*mask_array[i]+s_NPV[i]*x[i][3]*mask_array[i] for i in I), GRB.MAXIMIZE)
m7.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 1998218 nonzeros
Model fingerprint: 0x5ec884b1
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+05]
  Objective range  [1e-02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+06]
Found heuristic solution: objective 206364.03754
Presolve removed 120008 rows and 487705 columns
Presolve time: 0.76s
Presolved: 6535 rows, 18167 columns, 50205 nonzeros
Variable types: 0 continuous, 18167 integer (0 binary)

Root relaxation: objective 1.404570e+06, 6606 iterations, 2.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1404569.69    0    1 206364.038 1404569.69   581%     -    3s
H    0     0                

In [317]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m7.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m7.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_1M.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/3369593645.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  solution_gdf.to_file('solution_1M.shp')


In [331]:
solution_1M = geopandas.read_file('solution_1M.shp')
solution_1M.to_csv('solution_1M.csv')

In [318]:
#cost minimisation model - different equipment

I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=100000
m3=gp.Model()
x = [[m3.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m3.addConstr(((x[i][1])*0.8) <= loc_area[i])
for z in Z:
    for s in S:
        m3.addConstr(gp.quicksum((energy_array_seasonal[i][1][s]*x[i][1])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m3.addConstr(gp.quicksum( n_cost[i]*x[i][1] for i in I)<=cost_max)        
m3.setObjective(gp.quicksum(n_NPV[i]*x[i][1]*mask_array[i] for i in I), GRB.MAXIMIZE)
m3.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 505872 nonzeros
Model fingerprint: 0x25a0e76b
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-02, 2e+05]
  Objective range  [1e-02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+05]
Found heuristic solution: objective 33521.996726
Presolve removed 126542 rows and 499081 columns
Presolve time: 0.30s
Presolved: 1 rows, 6791 columns, 6791 nonzeros
Variable types: 0 continuous, 6791 integer (0 binary)

Root relaxation: objective 5.102765e+05, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 510276.521    0    1 33521.9967 510276.521  1422%     -    0s
H    0     0                    509339

In [319]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m3.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m3.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_NordexOnly.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/2932157339.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  solution_gdf.to_file('solution_NordexOnly.shp')


In [332]:
solution_NordexOnly = geopandas.read_file('solution_NordexOnly.shp')
solution_NordexOnly.to_csv('solution_NordexOnly.csv')

In [320]:
#cost minimisation model - different equipment, Solar PV

I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=100000
m8=gp.Model()
x = [[m8.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m8.addConstr(((x[i][3])*0.02) <= loc_area[i])
for z in Z:
    for s in S:
        m8.addConstr(gp.quicksum((energy_array_seasonal[i][3][s]*x[i][3])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m8.addConstr(gp.quicksum( s_cost[i]*x[i][3] for i in I)<=cost_max)        
m8.setObjective(gp.quicksum(s_NPV[i]*x[i][3]*mask_array[i] for i in I), GRB.MAXIMIZE)
m8.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 480602 nonzeros
Model fingerprint: 0x0deba520
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+05]
  Objective range  [3e+02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+05]
Found heuristic solution: objective -0.0000000

Explored 0 nodes (0 simplex iterations) in 0.14 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


In [321]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m8.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m8.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_SolarOnly.shp')

ValueError: Cannot write empty DataFrame to file.

In [322]:
#cost minimisation model - different equipment, Enercon

I=range(len(df_expanded_sum))
E=range(4)
Z=range(len(demand_const.columns))
S=range(2)
cost_max=100000
m9=gp.Model()
x = [[m9.addVar(vtype=GRB.INTEGER) for e in E] for i in I] #binary variables
for i in I:
    m9.addConstr(((x[i][2])*0.8) <= loc_area[i])
for z in Z:
    for s in S:
        m9.addConstr(gp.quicksum((energy_array_seasonal[i][2][s]*x[i][2])*loc_zones_array[i][z] for i in I) <=demand_const_array[z][s])      

m9.addConstr(gp.quicksum( e_cost[i]*x[i][2] for i in I)<=cost_max)        
m9.setObjective(gp.quicksum(e_NPV[i]*x[i][2]*mask_array[i] for i in I), GRB.MAXIMIZE)
m9.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126543 rows, 505872 columns and 505872 nonzeros
Model fingerprint: 0x94a3db4b
Variable types: 0 continuous, 505872 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-02, 2e+05]
  Objective range  [1e-01, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+05]
Found heuristic solution: objective 2013.8036382
Presolve removed 126542 rows and 505859 columns
Presolve time: 0.26s
Presolved: 1 rows, 13 columns, 13 nonzeros
Found heuristic solution: objective 121273.93254
Variable types: 0 continuous, 13 integer (0 binary)

Root relaxation: objective 2.136584e+05, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 213658.424    0    1 121273.933 213658.424  76.2%     -  

In [323]:
#append model results (location of turbines) to coordinates

solution_df=df_expanded_sum.copy()
vars=m9.getVars()
Vestas_Location=np.array([])
Nordex_Location=np.array([])
Enercon_Location=np.array([])
SolarPV_Location=np.array([])
for v in range(m9.NumVars):
    if v % 4 == 0:
        Vestas_Location=np.append(Vestas_Location,vars[v].x)
    else:
        if v % 4 == 1:
            Nordex_Location=np.append(Nordex_Location,vars[v].x)
        else:
            if v % 4 == 2:
                Enercon_Location=np.append(Enercon_Location,vars[v].x)
            else:
                SolarPV_Location=np.append(SolarPV_Location,vars[v].x)
            
solution_df['Vestas_Location']=pd.Series(Vestas_Location)
solution_df['Nordex_Location']=pd.Series(Nordex_Location)
solution_df['Enercon_Location']=pd.Series(Enercon_Location)
solution_df['SolarPV_Location']=pd.Series(SolarPV_Location)
solution_df=solution_df.loc[(solution_df['Vestas_Location'] != -0) | (solution_df['Nordex_Location'] != -0) | (solution_df['Enercon_Location'] != 0) | (solution_df['SolarPV_Location'] != 0)]
solution_gdf=geopandas.GeoDataFrame(solution_df, geometry='geometry')
solution_gdf.to_file('solution_EnerconOnly.shp')

/var/folders/w6/qlqct9bn2_b5pskgsph3f7hh0000gn/T/ipykernel_15356/1249264601.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  solution_gdf.to_file('solution_EnerconOnly.shp')
